In [1]:
#Data Cleaning and Organizing
import pandas as pd
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore')
import gc


#pre processing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

#Data Visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#Modeling
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import r2_score, mean_squared_error, f1_score, plot_confusion_matrix, accuracy_score, precision_score, recall_score

In [2]:
#each data set, not included in github
buildMetData = pd.read_csv('energyCSV/building_metadata.csv')
test = pd.read_csv('energyCSV/test.csv')
train = pd.read_csv('energyCSV/train.csv')
weatherTest = pd.read_csv('energyCSV/weather_test.csv')
weatherTrain = pd.read_csv('energyCSV/weather_train.csv')

train

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.000
1,1,0,2016-01-01 00:00:00,0.000
2,2,0,2016-01-01 00:00:00,0.000
3,3,0,2016-01-01 00:00:00,0.000
4,4,0,2016-01-01 00:00:00,0.000
...,...,...,...,...
20216095,1444,0,2016-12-31 23:00:00,8.750
20216096,1445,0,2016-12-31 23:00:00,4.825
20216097,1446,0,2016-12-31 23:00:00,0.000
20216098,1447,0,2016-12-31 23:00:00,159.575


In [3]:
# Function to reduce the DF size
# Code from Kaggle user Koustav Banerjee

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

weatherTrain = reduce_mem_usage(weatherTrain)
weatherTest = reduce_mem_usage(weatherTest)
buildMetData = reduce_mem_usage(buildMetData)

Mem. usage decreased to 289.19 Mb (53.1% reduction)
Mem. usage decreased to 596.49 Mb (53.1% reduction)
Mem. usage decreased to  3.07 Mb (68.1% reduction)
Mem. usage decreased to  6.08 Mb (68.1% reduction)
Mem. usage decreased to  0.03 Mb (60.3% reduction)


## Pre-processing

In [5]:
print(train.info(), test.info(), weatherTrain.info(), weatherTest.info(), buildMetData.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
 #   Column         Dtype  
---  ------         -----  
 0   building_id    int16  
 1   meter          int8   
 2   timestamp      object 
 3   meter_reading  float32
dtypes: float32(1), int16(1), int8(1), object(1)
memory usage: 289.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41697600 entries, 0 to 41697599
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   row_id       int32 
 1   building_id  int16 
 2   meter        int8  
 3   timestamp    object
dtypes: int16(1), int32(1), int8(1), object(1)
memory usage: 596.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   site_id             139773 non-null  int8   
 1   timestamp           139773 non-null  object 


The 'timestamp' coloumns in 'train', 'test', 'weatherTrain', 'weatherTest' - as well as the column 'primary_use' in BuildMetData - are currently an onject type. Need to change those formats into something more usable.

In [6]:
train['timestamp'] = pd.to_datetime(train['timestamp'])
test['timestamp'] = pd.to_datetime(test['timestamp'])

weatherTrain['timestamp'] = pd.to_datetime(weatherTrain['timestamp'])
weatherTest['timestamp'] = pd.to_datetime(weatherTest['timestamp'])

#buildMetData['primary_use'] = buildMetData['primary_use'].astype('category')

In [7]:
print(train.info(), test.info(), weatherTrain.info(), weatherTest.info(), buildMetData.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
 #   Column         Dtype         
---  ------         -----         
 0   building_id    int16         
 1   meter          int8          
 2   timestamp      datetime64[ns]
 3   meter_reading  float32       
dtypes: datetime64[ns](1), float32(1), int16(1), int8(1)
memory usage: 289.2 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41697600 entries, 0 to 41697599
Data columns (total 4 columns):
 #   Column       Dtype         
---  ------       -----         
 0   row_id       int32         
 1   building_id  int16         
 2   meter        int8          
 3   timestamp    datetime64[ns]
dtypes: datetime64[ns](1), int16(1), int32(1), int8(1)
memory usage: 596.5 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139773 entries, 0 to 139772
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------

Much better. Now, as the data is currently 'timestamp presents itself as a single moment in time. A single hour for every day. this can be broken into further cartegories: 'hour', 'day', 'weekday', & 'month'. This recategorizing could help us find trends in specific, months, days, or even hours for the data to find trends in.

In [8]:
train

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.000000
1,1,0,2016-01-01 00:00:00,0.000000
2,2,0,2016-01-01 00:00:00,0.000000
3,3,0,2016-01-01 00:00:00,0.000000
4,4,0,2016-01-01 00:00:00,0.000000
...,...,...,...,...
20216095,1444,0,2016-12-31 23:00:00,8.750000
20216096,1445,0,2016-12-31 23:00:00,4.825000
20216097,1446,0,2016-12-31 23:00:00,0.000000
20216098,1447,0,2016-12-31 23:00:00,159.574997


In [9]:
train["hour"] = train["timestamp"].dt.hour
train["day"] = train["timestamp"].dt.day
train["weekend"] = train["timestamp"].dt.weekday
train["month"] = train["timestamp"].dt.month

test["hour"] = test["timestamp"].dt.hour
test["day"] = test["timestamp"].dt.day
test["weekend"] = test["timestamp"].dt.weekday
test["month"] = test["timestamp"].dt.month

In [10]:
train

,building_id,meter,timestamp,meter_reading,hour,day,weekend,month
0,0,0,2016-01-01 00:00:00,0.000000,0,1,4,1
1,1,0,2016-01-01 00:00:00,0.000000,0,1,4,1
2,2,0,2016-01-01 00:00:00,0.000000,0,1,4,1
3,3,0,2016-01-01 00:00:00,0.000000,0,1,4,1
4,4,0,2016-01-01 00:00:00,0.000000,0,1,4,1
...,...,...,...,...,...,...,...,...
20216095,1444,0,2016-12-31 23:00:00,8.750000,23,31,5,12
20216096,1445,0,2016-12-31 23:00:00,4.825000,23,31,5,12
20216097,1446,0,2016-12-31 23:00:00,0.000000,23,31,5,12
20216098,1447,0,2016-12-31 23:00:00,159.574997,23,31,5,12


Neat-o. Next, concatenating train & test on BuildMetData and the respective weather df so that train and test have all the infomation to train on.

#Merging Train, BuildMetData, weatherTrain
trainBuild = train.merge(buildMetData, on=['building_id'], how='left')
train = trainBuild.merge(weatherTrain, on=['site_id','timestamp'], how='left')

#Merging Test, BuildMetData, weatherTest
testBuild = test.merge(buildMetData, on=['building_id'], how='left')
test = testBuild.merge(weatherTest, on=['site_id','timestamp'], how='left')

In [11]:
#merging for building metadata
temp_df = train[['building_id']]
temp_df = temp_df.merge(buildMetData, on=['building_id'], how='left')
del temp_df['building_id']
train = pd.concat([train, temp_df], axis=1)

temp_df = test[['building_id']]
temp_df = temp_df.merge(buildMetData, on=['building_id'], how='left')
del temp_df['building_id']
test = pd.concat([test, temp_df], axis=1)

del temp_df, buildMetData

#merging for wather data
temp_df = train[['site_id','timestamp']]
temp_df = temp_df.merge(weatherTrain, on=['site_id','timestamp'], how='left')

del temp_df['site_id'], temp_df['timestamp']
train = pd.concat([train, temp_df], axis=1)

temp_df = test[['site_id','timestamp']]
temp_df = temp_df.merge(weatherTest, on=['site_id','timestamp'], how='left')

del temp_df['site_id'], temp_df['timestamp']
test = pd.concat([test, temp_df], axis=1)

del temp_df, weatherTrain, weatherTest

In [12]:
train

,building_id,meter,timestamp,meter_reading,hour,day,weekend,month,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.000000,0,1,4,1,0,Education,7432,2008.0,NaN,25.000000,6.0,20.000000,NaN,1019.5,0.0,0.000000
1,1,0,2016-01-01 00:00:00,0.000000,0,1,4,1,0,Education,2720,2004.0,NaN,25.000000,6.0,20.000000,NaN,1019.5,0.0,0.000000
2,2,0,2016-01-01 00:00:00,0.000000,0,1,4,1,0,Education,5376,1991.0,NaN,25.000000,6.0,20.000000,NaN,1019.5,0.0,0.000000
3,3,0,2016-01-01 00:00:00,0.000000,0,1,4,1,0,Education,23685,2002.0,NaN,25.000000,6.0,20.000000,NaN,1019.5,0.0,0.000000
4,4,0,2016-01-01 00:00:00,0.000000,0,1,4,1,0,Education,116607,1975.0,NaN,25.000000,6.0,20.000000,NaN,1019.5,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20216095,1444,0,2016-12-31 23:00:00,8.750000,23,31,5,12,15,Entertainment/public assembly,19619,1914.0,NaN,1.700195,NaN,-5.601562,-1.0,1008.5,180.0,8.796875
20216096,1445,0,2016-12-31 23:00:00,4.825000,23,31,5,12,15,Education,4298,NaN,NaN,1.700195,NaN,-5.601562,-1.0,1008.5,180.0,8.796875
20216097,1446,0,2016-12-31 23:00:00,0.000000,23,31,5,12,15,Entertainment/public assembly,11265,1997.0,NaN,1.700195,NaN,-5.601562,-1.0,1008.5,180.0,8.796875
20216098,1447,0,2016-12-31 23:00:00,159.574997,23,31,5,12,15,Lodging/residential,29775,2001.0,NaN,1.700195,NaN,-5.601562,-1.0,1008.5,180.0,8.796875


There are a couple things that should be considered as me move forward.
- a lot of the data in this set is still left as null, inf, or empty. This can be delt with later but the biggest offender is floor_count
- This data was given pre train-test split
- Data is to big to merge train and test to create a train-test split via sklearn
- Some of the data needs to be label encoded because models don't like strings
- Timestamp is no longer needed since we have split it into deeper, separate columns

In [13]:
#changing inf data types to nan and then dropping
#train.replace([np.inf, -np.inf], np.nan, inplace=True)
#test.replace([np.inf, -np.inf], np.nan, inplace=True)

In [14]:
#train = train.dropna()
#test = test.dropna()

In [18]:
train.to_pickle('train.pkl')
test.to_pickle('test.pkl')
   
del train, test
gc.collect()

train = pd.read_pickle('train.pkl')
test = pd.read_pickle('test.pkl')

train

,building_id,meter,timestamp,meter_reading,hour,day,weekend,month,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.000000,0,1,4,1,0,Education,7432,2008.0,NaN,25.000000,6.0,20.000000,NaN,1019.5,0.0,0.000000
1,1,0,2016-01-01 00:00:00,0.000000,0,1,4,1,0,Education,2720,2004.0,NaN,25.000000,6.0,20.000000,NaN,1019.5,0.0,0.000000
2,2,0,2016-01-01 00:00:00,0.000000,0,1,4,1,0,Education,5376,1991.0,NaN,25.000000,6.0,20.000000,NaN,1019.5,0.0,0.000000
3,3,0,2016-01-01 00:00:00,0.000000,0,1,4,1,0,Education,23685,2002.0,NaN,25.000000,6.0,20.000000,NaN,1019.5,0.0,0.000000
4,4,0,2016-01-01 00:00:00,0.000000,0,1,4,1,0,Education,116607,1975.0,NaN,25.000000,6.0,20.000000,NaN,1019.5,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20216095,1444,0,2016-12-31 23:00:00,8.750000,23,31,5,12,15,Entertainment/public assembly,19619,1914.0,NaN,1.700195,NaN,-5.601562,-1.0,1008.5,180.0,8.796875
20216096,1445,0,2016-12-31 23:00:00,4.825000,23,31,5,12,15,Education,4298,NaN,NaN,1.700195,NaN,-5.601562,-1.0,1008.5,180.0,8.796875
20216097,1446,0,2016-12-31 23:00:00,0.000000,23,31,5,12,15,Entertainment/public assembly,11265,1997.0,NaN,1.700195,NaN,-5.601562,-1.0,1008.5,180.0,8.796875
20216098,1447,0,2016-12-31 23:00:00,159.574997,23,31,5,12,15,Lodging/residential,29775,2001.0,NaN,1.700195,NaN,-5.601562,-1.0,1008.5,180.0,8.796875


In [19]:
#Label encoding
#primary use is required, the rest are to convert these from float to int
le = LabelEncoder()

train['primary_use'] = le.fit_transform(train['primary_use']).astype(np.int8)
test['primary_use'] = le.fit_transform(test['primary_use']).astype(np.int8)

In [20]:
#We don't need time stamp anymore
train = train.drop("timestamp", axis = 1)
test = test.drop("timestamp", axis = 1)

In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 19 columns):
 #   Column              Dtype  
---  ------              -----  
 0   building_id         int16  
 1   meter               int8   
 2   meter_reading       float32
 3   hour                int64  
 4   day                 int64  
 5   weekend             int64  
 6   month               int64  
 7   site_id             int8   
 8   primary_use         int8   
 9   square_feet         int32  
 10  year_built          float16
 11  floor_count         float16
 12  air_temperature     float16
 13  cloud_coverage      float16
 14  dew_temperature     float16
 15  precip_depth_1_hr   float16
 16  sea_level_pressure  float16
 17  wind_direction      float16
 18  wind_speed          float16
dtypes: float16(9), float32(1), int16(1), int32(1), int64(4), int8(3)
memory usage: 1.2 GB


In [22]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41697600 entries, 0 to 41697599
Data columns (total 19 columns):
 #   Column              Dtype  
---  ------              -----  
 0   row_id              int32  
 1   building_id         int16  
 2   meter               int8   
 3   hour                int64  
 4   day                 int64  
 5   weekend             int64  
 6   month               int64  
 7   site_id             int8   
 8   primary_use         int8   
 9   square_feet         int32  
 10  year_built          float16
 11  floor_count         float16
 12  air_temperature     float16
 13  cloud_coverage      float16
 14  dew_temperature     float16
 15  precip_depth_1_hr   float16
 16  sea_level_pressure  float16
 17  wind_direction      float16
 18  wind_speed          float16
dtypes: float16(9), int16(1), int32(2), int64(4), int8(3)
memory usage: 2.4 GB


In [23]:
#Taking a quick look at the full data before we start the model
#train.hist(figsize=(20,20), bins=20)

## Data Prep

In [24]:
# creating a list if feat columns and drop columns for later indexing in train-test split
featCols = ['building_id', 'primary_use', 'hour', 'day', 'weekend', 'month', 'meter', 'square_feet', 'year_built', 'air_temperature', 'cloud_coverage', 'dew_temperature']

dropCols = ['precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']

In [25]:
valTarget = train['meter_reading']
del train['meter_reading']

In [26]:
train = train.drop(dropCols + ["site_id", "floor_count"], axis = 1)
test = test.drop(dropCols + ["site_id", "floor_count","row_id"], axis = 1)

In [27]:
gc.collect()

120

In [28]:
num_folds = 5
kf = KFold(n_splits = num_folds, shuffle = False, random_state = 42)
error = 0
models = []
for i, (train_index, val_index) in enumerate(kf.split(train)):
    if i + 1 < num_folds:
        continue
    print(train_index.max(), val_index.min())
    
    #Splitting The DF Into Train Test Split
    train_X = train[featCols].iloc[train_index]
    val_X = train[featCols].iloc[val_index]
    train_y = valTarget.iloc[train_index]
    val_y = valTarget.iloc[val_index]
    
    #Training The Model
    lgb_train = lgb.Dataset(train_X, train_y > 0)
    lgb_eval = lgb.Dataset(val_X, val_y > 0)
    params = {
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': {'binary_logloss'},
            'learning_rate': 0.1,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq' : 5
            }
    gbm_class = lgb.train(params,
                lgb_train,
                num_boost_round=2000,
                valid_sets=(lgb_train, lgb_eval),
               early_stopping_rounds=20,
               verbose_eval = 20)
    
    #we did this so that we only consider positive vals and no dta leakage would be there
    lgb_train = lgb.Dataset(train_X[train_y > 0], train_y[train_y > 0]) 
    lgb_eval = lgb.Dataset(val_X[val_y > 0] , val_y[val_y > 0])
    params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'learning_rate': 0.5,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq' : 5
            }
    gbm_regress = lgb.train(params,
                lgb_train,
                num_boost_round=2000,
                valid_sets=(lgb_train, lgb_eval),
               early_stopping_rounds=20,
               verbose_eval = 20)
#     models.append(gbm)

    y_pred = (gbm_class.predict(val_X, num_iteration=gbm_class.best_iteration) > .5) *\
    (gbm_regress.predict(val_X, num_iteration=gbm_regress.best_iteration))
    error += np.sqrt(mean_squared_error(y_pred, (val_y)))/num_folds
    print(np.sqrt(mean_squared_error(y_pred, (val_y))))
    break
print(error)

16172879 16172880
[LightGBM] [Info] Number of positive: 14588424, number of negative: 1584456
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.109724 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1161
[LightGBM] [Info] Number of data points in the train set: 16172880, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.902030 -> initscore=2.219987
[LightGBM] [Info] Start training from score 2.219987
Training until validation scores don't improve for 20 rounds
[20]	training's binary_logloss: 0.180013	valid_1's binary_logloss: 0.179619
[40]	training's binary_logloss: 0.156394	valid_1's binary_logloss: 0.17163
[60]	training's binary_logloss: 0.1448	valid_1's binary_logloss: 0.171614
[80]	training's binary_logloss: 0.136717	valid_1's binary_logloss: 0.173615
Early stopping, best iteration is:
[63]	training's bi

In [29]:
sorted(zip(gbm_regress.feature_importance(), gbm_regress.feature_name()),reverse = True)

[(43, 'day'),
 (40, 'month'),
 (30, 'dew_temperature'),
 (19, 'air_temperature'),
 (18, 'square_feet'),
 (16, 'weekend'),
 (16, 'hour'),
 (14, 'building_id'),
 (7, 'meter'),
 (3, 'primary_use'),
 (2, 'year_built'),
 (2, 'cloud_coverage')]

In [30]:
gc.collect()

318

In [31]:
from tqdm import tqdm
i=0
res=[]
step_size = 50000
for j in tqdm(range(int(np.ceil(test.shape[0]/50000)))):
    
    res.append(np.expm1
               (
                   (gbm_class.predict
                    (test.iloc[i:i+step_size], num_iteration=gbm_class.best_iteration) > .5) *\
                       (gbm_regress.predict(test.iloc[i:i+step_size], num_iteration=gbm_regress.best_iteration))
               )
              )
    i+=step_size

100%|██████████| 834/834 [00:28<00:00, 29.52it/s]


In [32]:
res = np.concatenate(res)
pd.DataFrame(res).describe()

,0
count,4.169760e+07
mean,inf
std,NaN
min,-1.000000e+00
25%,2.044004e+107
50%,9.292703e+113
75%,2.478401e+289
max,inf


In [33]:
res

array([0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
       4.15824659e+116, 2.47840103e+289, 2.47840103e+289])